### 라이브러리 선언

In [1]:
import pandas as pd

### 데이터 불러오기

In [2]:
dataUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/kopo_channel_seasonality_new.csv"

In [6]:
selloutData = pd.read_csv(dataUrl)

### 1. 데이터 타입 통합

In [9]:
## regionid : str, product: str, yearweek: int 
## qty: float

In [10]:
selloutData.REGIONID = selloutData.REGIONID.astype(str)
selloutData.PRODUCT = selloutData.PRODUCT.astype(str)
selloutData.YEARWEEK = selloutData.YEARWEEK.astype(int)
selloutData.QTY = selloutData.QTY.astype(float)

### 2. 실적값 안에